In [1]:
# Meta --------------------------------------------------------------------
## Title:         Summary of Changes table
## Author:        Wonjun Choi
## Date Created:  Sept-9-2022
## Date Edited:   Oct-4-2022

## Dependency: numpy, pandas, tabula
#
# This code converts Change_of_summary pdf files into csv.
# This code only does minimal data cleaning. Extensive data cleaning is
# required after merging/building datasets.

# Issue found: ID is sometimes integer, sometimes float.
#              One ID was character: 653032B in 2011 file.
#
#              In some years merger&acquisition tables were
#              not in the form of dataframe. (discarded)
###########################################################################
import os
import numpy as np
import pandas as pd
from tabula.io import read_pdf

# Hardcoding part
import hardcoding
pagecontent_all = hardcoding.pagecontent_all

# functions
def combine_if_key_is_na_from_(where, table, key, columns, join_by=" "):
    """
    prototype code for fixing nonreg_add of 2007
    
    ex) "below" (then keep first)
    ID | ADDITION             ID |   ADDITION
    --------------            ------------------
    NA |     I           =>   NA | I AM IRON MAN
    NA |    AM                NA |   AM IRON MAN
    123|  IRON MAN            123|   IRON MAN
    
    ex) "above"
    ID | ADDITION             ID |   ADDITION
    --------------            ------------------
    123|     I           =>   123|       I
    NA |    AM                NA |     I AM
    NA |  IRON MAN            NA | I I AM IRON MAN***
        
    Another code will copy ID from below and drop duplicated.
    
    This is a bad code... should have been absorbing the above...
    
    input
    =====
    where: "above" or "below"
    key: if key is NA, combine columns from {where} if there are multiple NA's
         in key in a row, find the first non-NA
    join_by: for join_by.join(["a", "b"])
    """
    t = table.copy()
    if type(columns) is str:
        columns = [columns]
    
    for col in columns:
        for row in range(len(t)):
#            print(row)
            if pd.isna(t.loc[row,key]) is True:
                combine_list = [t.loc[row,col]]
                if where == "below":
                    row2 = row
                    while pd.isna(t.loc[row2,key]) is True:
                        combine_list.append(t.loc[row2+1,col])
                        row2 = row2 + 1
                if where == "above":
                    row2 = row
                    while pd.isna(t.loc[row2,key]) is True:
                        combine_list.append(t.loc[row2-1,col])
                        #print(t.loc[row2-1,col])
                        row2 = row2 - 1
                combine_list = [x if not pd.isna(x) else "" for x in combine_list]
                combine_list = [str(x) if type(x)==float else x for x in combine_list]
                if where == "above":
                    combine_list.reverse()
                    if len(combine_list)==3:
                        combine_list = combine_list[1:]  # ***see docstring
                    
                joined_value = join_by.join(combine_list).strip()
                t.loc[row,col] = joined_value
    return t


def fillna_from_(where, table, columns):
    """    
    Fill na by copying from above/below.

    ex) where="below":
    ID |   CITY               ID |     CITY
    --------------            ------------------
    NA |    NA         =>     12 |    Atlanta
    12 |  Atlanta             12 |    Atlanta

    input
    =====
    where: "below", "above"
    table: pandas dataframe
    columns: list of strings
    """
    t = table.copy()
    for col in columns:
        for row in range(len(t)):
            if pd.isna(t.loc[row,col]):
                if where == "above":
                    t.loc[row,col] = t.loc[row-1,col]
                elif where == "below":
                    t.loc[row,col] = t.loc[row+1,col]
    # print("filled na")
    return t

# ====================================================================================
def tidy_reg_del(tab):
    t = tab.copy()
    if year in [2008]:
        col = ['REASON FOR DELETION', 'HOSPITAL NAME', 'CITY', 'STATE']
        for c in col:
            t[c] = t[c].apply(lambda text: text.replace("\r"," "))        
    elif year in [2010]:
        t = t
    elif year in [2011]:
        t = combine_if_key_is_na_from_("below",t,'ID','NAME')
        t = fillna_from_('below',t,['ID','CITY','STATE','REASON FOR DELETION'])
        t.columns = ['ID','HOSPITAL NAME','CITY','STATE','REASON FOR DELETION']
    elif year in [2012]:
        t = t[1:].reset_index(drop=True)
        t.columns = ['ID', 'REASON FOR DELETION', 'HOSPITAL NAME', 'CITY',
                     'STATE']
        t = combine_if_key_is_na_from_('above',t,'ID',
                                       ['REASON FOR DELETION'],join_by=" ")
        t = fillna_from_('above',t,['ID','HOSPITAL NAME','CITY','STATE'])
        t = t[~t.ID.duplicated(keep='last')].reset_index(drop=True)
    elif year in [2013]:
        t.columns = ['ID','HOSPITAL NAME','CITY','STATE','REASON FOR DELETION']
        if t.iloc[0,4] == 'DELETION':
            t = t[1:].reset_index(drop=True)
        t = combine_if_key_is_na_from_("above", t, 'ID', ['REASON FOR DELETION','HOSPITAL NAME'], join_by=' ')
        t = fillna_from_("above",t, columns=['ID','CITY','STATE'])
        t = t[~t.ID.duplicated(keep='last')].reset_index(drop=True)
    elif year in [2014]:
        if t.columns[0] != 'ID':
            t = t.T.reset_index().T.reset_index(drop=True)
        adddel = 'DELETION'
        if t.columns[2] == 'Unnamed: 0':
            t.columns = ['ID','HOSPITAL NAME','CITY','STATE','REASON FOR '+adddel]
            if t.iloc[0,4] == adddel:
                t = t[1:].reset_index(drop=True)
            t = combine_if_key_is_na_from_("above", t, 'ID',['REASON FOR '+adddel,'HOSPITAL NAME'], join_by=' ')
            # only last word of CITY is valid...
            t['CITY'] = t['HOSPITAL NAME'].apply(lambda t: t.split(" ")[-1])
            t['HOSPITAL NAME'] = t['HOSPITAL NAME'].apply(lambda t: " ".join(t.split(" ")[:-1]))
        else:
            t.columns = ['ID','HOSPITAL NAME','CITY','STATE','REASON FOR '+adddel]
            if t.iloc[0,4] == adddel:
                t = t[1:].reset_index(drop=True)
            t = combine_if_key_is_na_from_("above", t, 'ID',['REASON FOR '+adddel,'HOSPITAL NAME'], join_by=' ')

        t = fillna_from_("above",t, columns=['ID','CITY','STATE'])
        for col in ['REASON FOR '+adddel,'HOSPITAL NAME', 'CITY']:
            t[col] = t[col].apply(lambda x: x.replace('\r',' '))
        t = t[~t.ID.duplicated(keep='last')].reset_index(drop=True)
        
    else:
        print("specify year in tidy_reg_del")
    
    return t

def tidy_reg_add(tab):
    t = tab.copy()
    if year in [2012]:
        if t.iloc[0,1] == 'ADDITION':
            t = t[1:].reset_index(drop=True)
        t.columns = ['ID','REASON FOR ADDITION','HOSPITAL NAME','CITY','STATE']
        t = combine_if_key_is_na_from_("above", t, 'ID', ['REASON FOR ADDITION','HOSPITAL NAME'], join_by=' ')
        t = fillna_from_("above",t, columns=['ID','CITY','STATE'])
        for col in ['REASON FOR ADDITION','HOSPITAL NAME', 'CITY']:
            t[col] = t[col].apply(lambda x: x.replace('\r',' '))
        t = t[~t.ID.duplicated(keep='last')].reset_index(drop=True)
    elif year in [2013]:
        adddel = 'ADDITION'
        t.columns = ['ID','HOSPITAL NAME','CITY','STATE','REASON FOR '+adddel]
        if t.iloc[0,4] == adddel:
            t = t[1:].reset_index(drop=True)
        t = combine_if_key_is_na_from_("above", t, 'ID', ['REASON FOR '+adddel,'HOSPITAL NAME'], join_by=' ')
        t = fillna_from_("above",t, columns=['ID','CITY','STATE'])
        t = t[~t.ID.duplicated(keep='last')].reset_index(drop=True)
    elif year in [2014]:
        adddel = 'ADDITION'
        t.columns = ['ID','HOSPITAL NAME','CITY','STATE','REASON FOR '+adddel]
        if t.iloc[0,4] == adddel:
            t = t[1:].reset_index(drop=True)
        t = combine_if_key_is_na_from_("above", t, 'ID',['REASON FOR '+adddel,'HOSPITAL NAME'], join_by=' ')
        t = fillna_from_("above",t, columns=['ID','CITY','STATE'])

    else:
        print("specify year in tidy_reg_add")
    return t

def tidy_nonreg_del(tab):
    t = tab.copy()
    if year in [2007]:
        t = combine_if_key_is_na_from_("below", t, "ID", "REASON FOR DELETION")
        t = fillna_from_("below", t, ['ID','HOSPITAL NAME', 'CITY', 'STATE'])
        t = t.loc[~t.duplicated('ID')].reset_index().drop('index',axis=1)
    
    elif year in [2008]:
        col = ['REASON FOR DELETION', 'HOSPITAL NAME', 'CITY', 'STATE']
        for c in col:
            t[c] = t[c].apply(lambda text: text.replace("\r"," "))
    
    elif year in [2009]:
        t = combine_if_key_is_na_from_("below", t, "ID", ["REASON FOR DELETION","HOSPITAL NAME"])
        t = fillna_from_("below", t, ["ID", "CITY", "STATE"])
        t = t.loc[~t.duplicated('ID')].reset_index().drop('index',axis=1)
        
    elif year in [2010]:
        t['CITY'] = t['HOSPITAL NAME CITY'].apply(lambda x: x.split(" ")[-1])
        t['HOSPITAL NAME'] = t['HOSPITAL NAME CITY'].apply(lambda x: " ".join(x.split(" ")[:-1]))
        t = t.drop(columns=['HOSPITAL NAME CITY'])
        
    elif year in [2012]:
        t.columns = ['ID','REASON FOR DELETION','HOSPITAL NAME','CITY','STATE']

        if t.iloc[0,1] == 'DELETION':
            t = t[1:].reset_index(drop=True)
        t = combine_if_key_is_na_from_("above", t, 'ID', ['REASON FOR DELETION','HOSPITAL NAME'], join_by=' ')
        t = fillna_from_("above",t, columns=['ID','CITY','STATE'])
        for col in ['REASON FOR DELETION','HOSPITAL NAME', 'CITY']:
            t[col] = t[col].apply(lambda x: x.replace('\r',' '))
        t = t[~t.ID.duplicated(keep='last')].reset_index(drop=True)
    elif year in [2013]:
        adddel = 'DELETION'
        t.columns = ['ID','HOSPITAL NAME','CITY','STATE','REASON FOR '+adddel]
        if t.iloc[0,4] == adddel:
            t = t[1:].reset_index(drop=True)
        t = combine_if_key_is_na_from_("above", t, 'ID', ['REASON FOR '+adddel,'HOSPITAL NAME'], join_by=' ')
        t = fillna_from_("above",t, columns=['ID','CITY','STATE'])
        for col in ['REASON FOR '+adddel,'HOSPITAL NAME', 'CITY']:
            t[col] = t[col].apply(lambda x: x.replace('\r',' '))
        t = t[~t.ID.duplicated(keep='last')].reset_index(drop=True)
    elif year in [2014]:
        if t.columns[0] != 'ID':
            t = t.T.reset_index().T.reset_index(drop=True)
        adddel = 'DELETION'
        if t.columns[1] == 'Unnamed: 0':
            t = t.drop(columns='NAME')
        t.columns = ['ID','HOSPITAL NAME','CITY','STATE','REASON FOR '+adddel]
        t = combine_if_key_is_na_from_("above", t, 'ID',
                                       ['REASON FOR '+adddel,'HOSPITAL NAME'],
                                       join_by=' ')
        t = fillna_from_("above",t, columns=['ID','CITY','STATE'])
        for col in ['REASON FOR '+adddel,'HOSPITAL NAME', 'CITY']:
            t[col] = t[col].apply(lambda x: x.replace('\r',' ') 
                                  if pd.isna(x)==False else x)
        t = t[~t.ID.duplicated(keep='last')].reset_index(drop=True)        
    
    return t

def tidy_nonreg_add(tab):
    """
    dd
    """
    t = tab.copy()
    if year in [2007]:  # year is a global variable. consider f(tab,year=year)
        t.columns = t.iloc[0]
        t = t[1:].reset_index(drop=True)
        t = combine_if_key_is_na_from_("below", t, "ID", "ADDITION")
        t = fillna_from_("below", t, ['ID','HOSPITAL NAME','CITY','STATE'])
        t = t.loc[~t.duplicated('ID')].reset_index().drop('index', axis=1)
        t.columns = ['ID', 'REASON FOR ADDITION', 'HOSPITAL NAME', 'CITY', 'STATE']
        
    elif year in [2008,2009]:
        t.columns = ['ID', 'REASON FOR ADDITION', 'HOSPITAL NAME', 'CITY', 'STATE']
        t['REASON FOR ADDITION'] = t['REASON FOR ADDITION'].apply(lambda x: x.replace("\r"," "))

        if year == 2009:
            t = combine_if_key_is_na_from_("below", t, "ID", "REASON FOR ADDITION")
            t = fillna_from_("below", t, ["ID", "CITY", "STATE", "HOSPITAL NAME"])
            t = t.loc[~t.duplicated('ID')].reset_index().drop('index',axis=1)
            t = t.drop(t[t.ID == "ID"].index)
            
    elif year in [2010]:
        t.columns = ['ID', 'REASON FOR ADDITION', 'HOSPITAL NAME', 'CITY', 'STATE']
        t = t[1:].reset_index(drop=True)
        def f(text):
            name = text.split('Newly Added')[-1].split('Status changed to')[-1]
            if name == 'nonregistered':
                reason, name = 'nonregistered', np.nan
                return reason, name
            elif len(name) == 0:
                reason, name = 'no thx', 'no thx'  # second page is ok...
                return reason, name
            else:
                reason = text.split(name)[0]
                name = name[1:]  # remove blank
                return reason, name
        for row in range(len(t)):
            reason, name = f(t.loc[row, 'REASON FOR ADDITION'])
            if not reason=='no thx':
                t.loc[row, 'REASON FOR ADDITION'] = reason
                t.loc[row, 'HOSPITAL NAME'] = name            
        t = combine_if_key_is_na_from_("above", t, "ID", "REASON FOR ADDITION")
        t = fillna_from_("above",t,["ID","HOSPITAL NAME", "CITY", "STATE"])
        
    elif year in [2011]:
        t.columns = ['ID', 'HOSPITAL NAME', 'CITY', 'STATE', 'REASON FOR ADDITION']
        
    elif year in [2012]:
        t.columns = ['ID','REASON FOR ADDITION','HOSPITAL NAME','CITY','STATE']
        if t.iloc[0,1] == 'ADDITION':
            t = t[1:].reset_index(drop=True)
        t = combine_if_key_is_na_from_("above", t, 'ID', ['REASON FOR ADDITION','HOSPITAL NAME'], join_by=' ')
        t = fillna_from_("above",t, columns=['ID','CITY','STATE'])
        for col in ['REASON FOR ADDITION','HOSPITAL NAME', 'CITY']:
            t[col] = t[col].apply(lambda x: x.replace('\r',' '))
        t = t[~t.ID.duplicated(keep='last')].reset_index(drop=True)
        
    elif year in [2013]:
        if t.columns[0] != 'ID':
            t = t.T.reset_index().T.reset_index(drop=True)
        adddel = 'ADDITION'
        t.columns = ['ID','HOSPITAL NAME','CITY','STATE','REASON FOR '+adddel]
        if t.iloc[0,4] == adddel:
            t = t[1:].reset_index(drop=True)
        t = combine_if_key_is_na_from_("above", t, 'ID',['REASON FOR '+adddel,'HOSPITAL NAME'], join_by=' ')
        t = fillna_from_("above",t, columns=['ID','CITY','STATE'])
        for col in ['REASON FOR '+adddel,'HOSPITAL NAME', 'CITY']:
            t[col] = t[col].apply(lambda x: x.replace('\r',' '))
        t = t[~t.ID.duplicated(keep='last')].reset_index(drop=True)\
        
    elif year in [2014]:
        adddel = 'ADDITION'
        if t.columns[1] == 'Unnamed: 0':
            t = t.drop(columns='NAME')
        t.columns = ['ID','HOSPITAL NAME','CITY','STATE','REASON FOR '+adddel]
      
    return t
    
def tidy_merger(tab):
    t = tab.copy()
    
    if year in [2007]:
        t = t[1:].reset_index(drop=True)

        t['Unnamed: 0'] = t['ID NAME'].apply(lambda idname: ' '.join(idname.split(' ')[1:]) 
                                             if pd.isna(idname)==False else np.nan)
        t['ID NAME'] = t['ID NAME'].apply(lambda idname: idname.split(' ')[0] 
                                          if pd.isna(idname)==False else np.nan)

        t['Unnamed: 1'] = t['MERGER MERGED NAME'].apply(lambda mmn: ' '.join(mmn.split(' ')[1:]) 
                                                        if pd.isna(mmn)==False else np.nan)
        t['MERGER MERGED NAME'] = t['MERGER MERGED NAME'].apply(lambda mmn: mmn.split(' ')[0] 
                                                                if pd.isna(mmn)==False else np.nan)

        t['MERGED STATE'] = t['MERGED CITY MERGED'].apply(lambda mcm: mcm.split(' ')[-1] 
                                                          if pd.isna(mcm)==False else np.nan)
        t['MERGED CITY MERGED'] = t['MERGED CITY MERGED'].apply(lambda mcm:' '.join(mcm.split(' ')[:-1]) 
                                                                if pd.isna(mcm)==False else np.nan)

        t = t.rename(columns = {'ID NAME': 'ID', 'Unnamed: 0': 'NAME',
                                'MERGER MERGED NAME': 'MERGER RESULT ID',
                           'Unnamed: 1': 'MERGED NAME', 'MERGED CITY MERGED': 'MERGED CITY'})
        t = t.reset_index().drop(['index'],axis=1)

        t = fillna_from_("above", t, ['MERGER RESULT ID', 'MERGED NAME', 'MERGED CITY', 
                                      'MERGED STATE'])
        
    elif year in [2008]:
        t['Unnamed: 0'] = t['ID HOSPITAL NAME'].apply(lambda idname: ' '.join(idname.split(' ')[1:]) 
                                                      if pd.isna(idname)==False else np.nan)
        t['ID HOSPITAL NAME'] = t['ID HOSPITAL NAME'].apply(lambda idname: idname.split(' ')[0] 
                                                            if pd.isna(idname)==False else np.nan)
        
    elif year in [2009]:
        t.columns = [x.replace("\r"," ") for x in t.columns]
        t = fillna_from_("above", t, ['MERGER RESULT ID', 'MERGED NAME', 'MERGED CITY', 
                                      'MERGED STATE'])

    return t


##########################################################################
# make pdf tables into csv

dir_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
outfile_base = os.path.join(dir_root, 'data','temp')

double_check = {}

for year in range(2015,2016):
    print('year {}'.format(year))
    file_pdf = os.path.join(dir_root,'data','input','AHA FY {}'.format(year),
                            *hardcoding.filenames[year])
    tables = read_pdf(file_pdf, pages='all', multiple_tables=True)
    
    pagecontent = pagecontent_all[str(year)]
    
    workingon = ["Reg Del","Reg Add","Nonreg Del","Nonreg Add"]
    for wo in workingon:
        print(wo)
        if wo == 'Reg Del':  # modify this if syntax later
            tidy_tables = [tidy_reg_del(tables[page]) for page in pagecontent[wo]]
        elif wo == 'Reg Add':
            tidy_tables = [tidy_reg_add(tables[page]) for page in pagecontent[wo]]
        elif wo == 'Nonreg Del':
            tidy_tables = [tidy_nonreg_del(tables[page]) for page in pagecontent[wo]]
        elif wo == 'Nonreg Add':
            tidy_tables = [tidy_nonreg_add(tables[page]) for page in pagecontent[wo]]
        double_check[wo] = tidy_tables  # for debug and double check
        table_csv = pd.concat(tidy_tables, ignore_index=True)
        table_csv = table_csv.dropna()
        table_csv.to_csv(os.path.join(outfile_base,
                                      'change_'+wo+'_{}.csv'.format(year)),
                        header=True, index=False)
        print(wo+" done")
        
    # if year in [2008,2010]:
    #     print("Merger and Acquisitions {} table is a disaster".format(year))
    #     pass
    # else:
    #     table_merger = pd.concat([tidy_merger(tables[page])
    #                               for page in pagecontent["Mergers and Acquisitions"]],
    #                               ignore_index=True)
    #     table_merger.to_csv(os.path.join(outfile_base,'change_merger_{}.csv'.format(year)),
    #                         header=True, index=False)
    # print("merger done")
    
print("I'm Done!!")

year 2015
Reg Del
specify year in tidy_reg_del
specify year in tidy_reg_del
Reg Del done
Reg Add
specify year in tidy_reg_add
Reg Add done
Nonreg Del
Nonreg Del done
Nonreg Add


IndexError: list index out of range

In [ ]:
# double_check["Reg Del"][3]
# double_check["Reg Add"][0]
# double_check["Nonreg Del"][1]
double_check["Nonreg Add"][0]

In [2]:
file_pdf = os.path.join(dir_root,'data','input','AHA FY {}'.format(year),
                            *hardcoding.filenames[year])
tables = read_pdf(file_pdf, pages='all', multiple_tables=True)

In [3]:
pagecontent

{'Reg Del': [1, 2],
 'Reg Add': [3],
 'Nonreg Del': [5],
 'Nonreg Add': [6, 7],
 'Mergers and Acquisitions': []}

In [4]:
# tidy snippet
t = tables[1].copy()

# if t.columns[0] != 'ID':
#     t = t.T.reset_index().T.reset_index(drop=True)

adddel = 'DELETION'

# idiosyncratic
# if t.columns[2] == 'Unnamed: 0':
#     t.columns = ['ID','HOSPITAL NAME','CITY','STATE','REASON FOR '+adddel]
#     if t.iloc[0,4] == adddel:
#         t = t[1:].reset_index(drop=True)
#     t = combine_if_key_is_na_from_("above", t, 'ID',['REASON FOR '+adddel,'HOSPITAL NAME'], join_by=' ')
#     # only last word of CITY is valid...
#     t['CITY'] = t['HOSPITAL NAME'].apply(lambda t: t.split(" ")[-1])
#     t['HOSPITAL NAME'] = t['HOSPITAL NAME'].apply(lambda t: " ".join(t.split(" ")[:-1]))
if t.columns[1] == 'Unnamed: 0':
    t = t.drop(columns='NAME')
#=====
t.columns = ['ID','HOSPITAL NAME','CITY','STATE','REASON FOR '+adddel]
# if t.iloc[0,4] == adddel:
#     t = t[1:].reset_index(drop=True)
    
# t = combine_if_key_is_na_from_("above", t, 'ID',
#                                ['REASON FOR '+adddel,'HOSPITAL NAME'],
#                                join_by=' ')
# t = fillna_from_("above",t, columns=['ID','CITY','STATE'])
# for col in ['REASON FOR '+adddel,'HOSPITAL NAME', 'CITY']:
#     t[col] = t[col].apply(lambda x: x.replace('\r',' ') 
#                           if pd.isna(x)==False else x)
# t = t[~t.ID.duplicated(keep='last')].reset_index(drop=True)

t

,ID,HOSPITAL NAME,CITY,STATE,REASON FOR DELETION
0,NaN,NaN,NaN,NaN,and Parent Organization for Merged Entities
1,6110173.0,Parkview Adventist Medical Center Brunswick,NaN,ME,Ambulatory Care Center
2,6141800.0,Spaulding Hospital for Continuing Medical Care...,NaN,MA,Closed
3,NaN,NaN,NaN,NaN,Merged into 6212925
4,6210770.0,NYU Lutheran Brooklyn,NaN,NY,"NYU Lagone Medical Center - New York, NY"
...,...,...,...,...,...
58,6740041.0,Central Texas Hospital Cameron,NaN,TX,Closed
59,6740252.0,East Texas Medical Center-Gilmer Gilmer,NaN,TX,Closed
60,6740275.0,Kindred Hospital North Houston Houston,NaN,TX,Closed
61,6740278.0,Kindred Hospital East Houston Channelview,NaN,TX,Closed


In [ ]:
?combine_if_key_is_na_from_

In [ ]:
        t.columns = ['ID', 'REASON FOR ADDITION', 'HOSPITAL NAME', 'CITY', 'STATE']
        t = t[1:].reset_index(drop=True)
        def f(text):
            name = text.split('Newly Added')[-1].split('Status changed to')[-1]
            if name == 'nonregistered':
                reason, name = 'nonregistered', ''
                return reason, name
            elif len(name) == 0:
                reason, name = 'no thx', 'no thx'  # second page is ok...
                return reason, name
            else:
                reason = text.split(name)[0]
                name = name[1:]  # remove blank
                return reason, name
        t['HOSPITAL NAME'] = t['REASON FOR ADDITION'].apply(lambda x: f(x)[1] if (f(x)[1] is not 'no thx') else x)
        t['REASON FOR ADDITION'] = t['REASON FOR ADDITION'].apply(lambda x: f(x)[0] if (f(x)[0] is not 'no thx') else x)
        t = combine_if_key_is_na_from_("below", t, "ID", "REASON FOR ADDITION")
        t = fillna_from_("above",t,["ID","HOSPITAL NAME", "CITY", "STATE"])

In [ ]:
def f(text):
        name = text.split('Newly Added')[-1].split('Status changed to')[-1]
        if name == 'nonregistered':
            reason, name = 'nonregistered', np.nan
            return reason, name
        elif len(name) == 0:
            reason, name = 'no thx', 'no thx'  # second page is ok...
            return reason, name
        else:
            reason = text.split(name)[0]
            name = name[1:]  # remove blank
            return reason, name

In [ ]:
asd = [tables[page] for page in pagecontent["Nonreg Add"]]
tt = asd[1].copy()
tt.columns = ['ID', 'REASON FOR ADDITION', 'HOSPITAL NAME', 'CITY', 'STATE']
tt = tt[1:].reset_index(drop=True)
tt['HOSPITAL NAME'] = tt['REASON FOR ADDITION'].apply(lambda x: f(x)[1] if (f(x)[1] is not 'no thx') else x)
# tt['REASON FOR ADDITION'] = tt['REASON FOR ADDITION'].apply(lambda x: f(x)[0] if (f(x)[0] is not 'no thx') else x)
#tt = combine_if_key_is_na_from_("above", tt, "ID", "REASON FOR ADDITION")

tt

In [ ]:
# table_reg_del[0:50]
# table_reg_del[51:100]
# table_reg_del[101:]

table_reg_add[0:50]
# table_reg_add[51:100]
# table_reg_add[101:]

# table_nonreg_del[0:50]
# table_nonreg_del[51:100]
# table_nonreg_del[101:]

# table_nonreg_add[0:50]
# table_nonreg_add[51:100]
# table_nonreg_add[101:]

#table_merger

In [ ]:
table_nonreg_del.loc[3,'HOSPITAL NAME'].replace("\r"," ")

In [ ]:
table_reg_del.loc[59,'ID']

In [ ]:
pagecontent["Nonreg Add"]
tables[11]